# Python 中級編1：クラスタリング
## 目次
* [K-means アルゴリズムの概要](#K-means-アルゴリズムの概要)
* [テストデータの作成](#テストデータの作成)
* [リスト内包記法](#リスト内包記法)
* [K-means アルゴリズムの実装（2次元バージョン）](#K-means-アルゴリズムの実装（2次元バージョン）)
 * [Step 0：初期化](#Step-0：初期化)
 * [Step 1: 重心の計算](#Step-1:-重心の計算)
 * [Step 2: クラスタへの再配置](#Step-2:-クラスタへの再配置)
 * [Step 3: メイン処理](#Step-3:-メイン処理)
  * [練習10.1：クラスタの数が合っていない場合](#練習10.1：クラスタの数が合っていない場合)
* [課題10.3：ベクトル量子化](#課題10.3：ベクトル量子化)
 * [画像の読み込み](#画像の読み込み)
 * [画像データとピクセル](#画像データとピクセル)
 * [画像の縮小](#画像の縮小)
 * [クラスタリングによる画像の圧縮](#クラスタリングによる画像の圧縮)
 * [練習10.4: ベクトル量子化全体の処理の関数化](#練習10.4:-ベクトル量子化全体の処理の関数化)
* [課題提出の前の注意](#課題提出の前の注意)

---
このノートブックでは，多数のデータを自動的にグループ化するアルゴリズムを題材として Python プログラミングの練習を行う．

そのようなアルゴリズムは一般的に「クラスタリング」と呼ばれる．
このノートブックではそのうち K-means アルゴリズム（K-平均法）と呼ばれる手法を用いる．

---

## K-means アルゴリズムの概要

K-means アルゴリズムは，$n$ 次元の実数ベクトル $x_1, x_2, \dots, x_N \in \mathbb{R}^n$ を入力とし，これらを排他的な $K$ 個の部分集合 $C_1, C_2, \dots, C_K$ に分割する．すなわち
$$
C_1 \cup C_2 \cup \cdots \cup C_K = \{x_1, x_2, \dots, x_N\}
$$
かつ
$$
C_i \cap C_j = \emptyset \;\; (i \ne j)
$$
である．それぞれの部分集合 $C_i \;(i = 1, 2, \dots, K)$ のことを「クラスタ」と呼ぶ．

このような分割のしかたは非常にたくさんあるが，K-means アルゴリズムはそのうち，以下のような意味で「まとまりのよい」クラスタを見つけようとする．まず，各クラスタ $C_i$ に対して，その重心 $c_i$ を
$$
c_i = \frac{1}{|C_i|}\sum_{x \in C_i} x
$$
と定義する．ここで $|C_i|$ はクラスタの大きさ（含まれるベクトルの数）を表す．すなわち重心 $c_i$ はクラスタ $C_i$ に属するベクトルの平均である<sup>注1</sup>．

K-means アルゴリズムが見つけようとするのは以下のような分割である：「任意の $C_i$ および任意の $x \in C_i$ に対して，$x$ は全クラスタの重心 $c_1, c_2, \dots, c_K$ のうち自分が属するクラスタの重心 $c_i$ に最も近い」．そのような分割をここでは「安定な分割」と呼ぶことにする．

K-means アルゴリズムは，以下のような処理で安定な分割を見つけようとする：

0. 入力データをクラスタ群 $C_1, \dots, C_K$ にランダムに分割する
1. 各クラスタについて，その重心 $c_1, \dots, c_K$ を計算する
2. 各入力データを，最も近い重心をもつクラスタに（一斉に）移動させる
3. もしもクラスタ間でのデータの移動があったら 1. に戻る．移動がなければアルゴリズム終了

この繰り返しによって，ある意味でクラスタは段々「まとまりがよくなっていく」ことが証明できる（ここでは省略）．データの移動がなくなってアルゴリズムが終了したときには，安定な分割が得られていることは明らかだろう．

<sup>注1</sup> データ $x_1, \dots, x_N$ には重複があってもよい．よって正確には各クラスタ $C_1, \dots, C_K$ も重複を許した集合（multiset）である．しかし説明の簡単さのため集合の記号を流用する．気になる人はデータには重複はないものと思って読めばよい．

## テストデータの作成
まず2次元ベクトルに対する K-means アルゴリズムを実装する．そのテストのために，もともとある程度のまとまりがあるデータを人工的に作っておく．

以下のセルを実行すると，4つの等方的な2次元正規分布から多数の点がランダムに生成される．等方的というのは生成される点の $x$ 成分と $y$ 成分に相関がなく，かつそれぞれの分散が等しいということである．分布を式で書けば 

$$
f(x,y) = \frac{1}{2\pi\sigma^2}\exp\left(-\frac{(x-\mu_1)^2 + (y-\mu_2)^2}{2\sigma^2}\right)
$$

となる．下のセルでは4つの分布それぞれの平均 $(\mu_1, \mu_2)$ を $(0, 0), (2, 0), (0.5, 1.5), (2.0, 2.0)$ とし，いずれも分散 $\sigma^2 = 0.25$ としている．$x$ と $y$ は独立なので，それぞれを平均 $\mu_1$ および $\mu_2$，分散 $\sigma^2$ の正規分布から生成すればよい：

In [ ]:
import random

# ４つの正規分布の平均 (μ1, μ2)
means = [(0.0, 0.0), (2.0, 0.0), (0.5, 1.5), (2.0, 2.0)]

# 各分布の標準偏差 σ
stds = [0.5, 0.5, 0.5, 0.5]

# 各分布から生成する点の数
nums = [200, 100, 200, 100]

# 各分布から生成された点のリストのリスト
true_clusters = []
for (cx ,cy), std, num in zip(means, stds, nums):
    cluster = []
    for _ in range(num):
        # x 成分，y 成分をそれぞれ正規分布から生成
        x = random.gauss(cx, std)
        y = random.gauss(cy, std)
        cluster.append((x, y))
    true_clusters.append(cluster)

各点（ベクトル）はタプルで `(x, y)` のように表され，それぞれの正規分布から生成された点の集合は `[(x1, y1), (x2, y2), ...]` というタプルのリストで表されている．それらをさらにリストにした2重リストが上のセルで作られる `true_clusters` になる．

どんなデータになったか，元になった正規分布ごとに色分けした散布図で見てみよう（下のコードの細かい所を気にする必要はない）．黒い×印は各分布の平均 $(\mu_1, \mu_2)$ を表す．

In [ ]:
import matplotlib.pyplot as plt
# %matplotlib inline # 不要 & 使うと後でグラフが表示されないことがある

def show_2d_clusters(clusters, centers=None):
    # 縦横の比を 1:1 にする
    plt.gca().set_aspect("equal")
    
    for cluster in clusters:
        # (x,y)のリストを x のリストと y のリストに分ける
        xs = []
        ys = []
        for x, y in cluster:
            xs.append(x)
            ys.append(y)

        # 散布図として表示
        plt.scatter(xs, ys, marker=".")
        
    if centers != None: # 重心が入力されていたら表示
        for i in range(len(centers)):
            if clusters[i] != []: # クラスタが空でないときだけ表示
                cx, cy = centers[i]
                plt.plot(cx, cy, color="black", marker="x")
                
    plt.show()
        
show_2d_clusters(true_clusters, means)

データの各点が「どの分布から生成された点か」という情報を与えずに，点たちの座標のみから，だいたい上の図のようなグループ分けを復元することが目標である．


クラスタリングの入力とするために，全ての点をひとつのリストにして，ついでに順番をランダムに並べ替えておこう：

In [ ]:
# 各分布からの点のリストを連結
data2d = []
for clusters in true_clusters:
    data2d += clusters
    
# 順番をランダムに入れ替える
random.shuffle(data2d)

できた `data2d` をいちおう図示してみよう：

In [ ]:
show_2d_clusters([data2d])

## リスト内包記法
アルゴリズムの実装に取り掛かるまえに，**リスト内包記法**について説明する．必須の知識ではないが，これを使うと繰り返し処理が短く書けるため，今回のような課題には便利である．

例えば，リスト `xs = [1, 2, 3]` に対して，各要素を2倍した `two_xs = [2, 4, 6]` というリストは以下のように作れる：

In [ ]:
xs = [1, 2, 3]

two_xs = [2 * x for x in xs] # これがリスト内包記法

print(two_xs)

これまでに学習したやり方で書けば，同じことは以下のように書ける：

In [ ]:
xs = [1, 2, 3]

two_xs = []
for x in xs: # xs の各要素 x に対して
    two_xs.append(2 * x) # 要素を２倍したものを two_xs に追加する
    
print(two_xs)

リスト内包記法の一般形は以下のようになる：
```python
[ <繰り返し変数を使った式> for <繰り返し変数> in <リストまたはrange> ]
```
上の `two_xs = [2 * x for x in xs]` という例では，
* `<繰り返し変数を使った式>` が `2 * x`
* `<繰り返し変数>` が `x`
* `<リストまたはrange>` が `xs` である．

より一般的な（複雑な）書き方もできるが，ここではやらない．

**ミニ練習1**: 名前のリスト `names` を入力として，それぞれの名前の前に `'こんにちは，'` を付加した文字列のリストを返す関数 `hello(names)` を，**リスト内包記法を使って**実装せよ．例えば入力が `['太郎', '花子']` ならば，出力は `['こんにちは，太郎', 'こんにちは，花子']` となる：

In [ ]:
def hello(names):
    # *** 実装しなさい ***
    
# テスト
print(hello(['太郎', '花子']))

**ミニ練習2**: まず入力された数 `x` に対してその2乗を返す関数 `square(x)` を実装しなさい．さらに，数のリスト `xs` を入力とし，それぞれの数を2乗したもののリストを返す関数 `square_list(xs)` を `square(x)` と**リスト内包記法を使って**実装しなさい．

In [ ]:
def square(x):
    # *** 実装しなさい ***
    
def square_list(xs):
    # *** 実装しなさい ***
    
# テスト
square_list([1, 2, 3, 4]) == [1, 4, 9, 16]

以下では，何度か「$N$ 個の空のリストからなるリスト」を作る必要がある．これはリスト内包記法を使うと次のように書ける：

In [ ]:
N = 4
empties = [ [] for _ in range(N) ]

print(empties)

`<繰り返し変数>` のところ（`for` と `in` の間）が `_` となっているのは，`in range(N)` によって $N$ 回の繰り返しを行うが，$0, 1, ..., N-1$ と変化する繰り返し変数の値じたいは使わない，という意味（意思表示）である．別に普通の名前をつけてもよい．

for 文を使って書けば，上の `empties` の作り方は以下と同じ意味になる：

In [ ]:
N = 4
empties = []
for _ in range(N):
    empties.append([])
    
print(empties)

## K-means アルゴリズムの実装（2次元バージョン）
それでは，まず2次元の点に対する K-means アルゴリズムを実装する．もう一度アルゴリズムを箇条書きにしておく．カッコ内は，この後それぞれのステップを実装する関数名である：

0. 入力データをクラスタ群 $C_1, \dots, C_K$ にランダムに分割する（`random_split`）
1. 各クラスタについて，その重心 $c_1, \dots, c_K$ を計算する（`mean`）
2. 各入力データを，最も近い重心をもつクラスタに（一斉に）移動させる（`reallocate`）
3. もしもクラスタ間でのデータの移動があったら 1. に戻る．移動がなければアルゴリズム終了（`k_means`）

### Step 0：初期化
2次元の点 `(x, y)` のリスト `data = [(x1, y1), (x2, y2), ...]` を受け取り，点を `K` 個のクラスタにランダムに分ける関数 `random_split(data, K)` を実装しなさい．

各クラスタは点を表すタプル `(x, y)` のリストで表し，出力である `K` 個のクラスタは，クラスタのリストとして返す
（よって，出力全体は「`(x, y)` のリストのリスト」で2重リストになる）．

これ以降も同様に，クラスタは「点 `(x, y)` のリスト」で表し，クラスタの集合は「点 `(x, y)` のリストのリスト」で表す．

入出力例（乱数を使うので必ずこうなる訳ではない）：
```python
入力：data = [(1, 1), (2, 2), (3, 3), (4, 4)], K = 2

出力：random_split(data, K) 
     --> [ [(2, 2), (3, 3)],  [(1, 1), (4, 4)] ]
```

ヒント（ダブルクリックで表示）
<!--
* まず，K 個の空のリストを要素とするリスト clusters = [[], [], ...] を作る．
  上で練習したリスト内包記法を使うと簡単にできる．

* data の各要素 v に対して 0, 1, ..., K-1 の中からランダムに一つ数を選びそれを i とする．
  A ≦ x ≦ B の範囲の整数をランダムに得るのは random.randint(A, B) でできる．

* 選んだ i を用いて, v を clusters[i] に append する
-->

In [ ]:
def random_split(data, K):
    # *** 実装しなさい ***


実装できたら，2つ前のセクションで作った2次元の点のデータ `data2d` を4つのクラスタへとランダムに分割し，同じセクションで定義した `show_2d_clusters` を使って色分けして表示してみよう：

In [ ]:
initial_clusters = random_split(data2d, 4)
show_2d_clusters(initial_clusters)

４色がごちゃごちゃになっていれば，多分正しく実装できている．

### Step 1: 重心の計算
点を表すタプル `(x, y)` のリストとして表された（ひとつの）クラスタ`cluster = [(x1, y1), (x2, y2), ...]` を受け取り，その重心（平均）をタプル `(mx, my)` として返す関数 `mean` を実装しなさい．

ヒント（ダブルクリックで表示）
<!--
* まず x 座標の和と y 座標の和を入れる変数をひとつずつ用意する

* cluster の中の各点 (x, y) に対するループは
  
  for x, y in clusters:
      ...

  という形で書けば x 座標と y 座標がループ変数として取り出せる

* ループで x 座標と y 座標それぞれの和を計算したら, cluster 内の点の数 len(clusters) で
  座標値の和を割って平均値を得る

* 最後に

  return (x座標の平均, y座標の平均)

  で平均値のタプルを返す
-->

In [ ]:
def mean(cluster):
    # *** 実装しなさい ***


実装したらテストしなさい：

In [ ]:
print(mean([(1, 4), (2, 8), (3, 6), (4, 2)]) == (2.5, 5.0))
print(mean([(10, 20)]) == (10, 20)) # 特殊ケース：要素がひとつだけのクラスターの場合

実はアルゴリズムの途中でクラスタが空集合になってしまう場合が出てくる．その場合にも `mean` からは何か返しておかないと他の処理が面倒になる．

いったん空になったクラスタにはそれ以降も点を割り当てなければよいので，空のクラスタ（すなわち `[]`）を受け取ったら「とても遠くの点」つまり x, y 座標が両方 $\infty$ の点を返すように `mean` を修正しよう．

「$\infty$」は `float("inf")` で作れる．

入力として `[]` を受け取ったら $(\infty, \infty)$ すなわち python の書き方では `( float("inf"), float("inf") )` を返すように `mean` を修正しなさい．入力が `[]` かどうかは
```python
if cluster == []:
    ...
```
という場合わけで分かる．

修正したら，いちおうテストしましょう：

In [ ]:
# 空のクラスタが入力された場合
print(mean([]) == (float("inf"), float("inf")))

# 空でない場合
print(mean([(1, 4), (2, 8), (3, 6), (4, 2)]) == (2.5, 5.0))
print(mean([(10, 20)]) == (10, 20)) # 特殊ケース：要素がひとつだけのクラスターの場合

### Step 2: クラスタへの再配置
$K$ 個のクラスタ $C_1, \dots, C_K$ の重心 $c_1, \dots, c_K$ が計算できたら，まず各データ点 $v = (x, y)$ について最も近い重心を探す必要がある．

最も近い重心が $c_i$ だとすると，$v$ は次にクラスタ $C_i$ に再配置される．この処理のために，最も近い重心 $c_i$ そのものではなく，その番号 `i` を探したい．

重心のリスト `centers = [(cx1, cy1), (cx2, cy2), ...]` とデータ点 `v = (x, y)` を受け取り，`v` に最も近い重心の添え字（0 から始まることに注意）を返す関数 `find_closest(centers, v)` を実装しなさい：

ヒント（ダブルクリックで表示）
<!--
* それまでに見つかった最も近い重心の番号を保存する変数 min_i と，
  その重心への v からの距離を保存する変数 min_distance を用意する．

* min_distance は「とても大きい数」である float("inf") で初期化する．

* あとは centers の点を一つずつ調べ, v からの距離が min_distance よりも
  小さかったら, min_i と min_distance を更新する．
 （最初のクラスタの中心については, 必ず min_i と min_distance の更新が起きる）

* 最後に min_i を return する
-->

In [ ]:
def find_closest(centers, v):
    # *** 実装しなさい ***


実装できたらテストしなさい：

In [ ]:
print(find_closest([(1, 2), (3, 4), (5, 6)], (1.001, 2.001)) == 0)
print(find_closest([(1, 2), (3, 4), (5, 6)], (3.001, 4.001)) == 1)
print(find_closest([(1, 2), (3, 4), (5, 6)], (5.001, 6.001)) == 2)

もっとも近い重心（の番号）が分かれば，再配置の処理は難しくない．

再配置処理を行う以下のような関数 `reallocate(centers, clusters)` を実装しなさい：
* 入力として，重心のリスト `centers = [(cx1, cy1), (cx2, cy2), ..., (cxK, cyK)]` と，現在のクラスタのリスト `clusters` を受け取る
* 出力として，以下で定義する `new_clusters` と `change` をまとめてリスト `[new_clusters, change]` として返す：
  * `new_clusters`: 最も近い重心をもつクラスタにデータ点を再配置したクラスタ集合
  * `change`: 再配置の間にクラスタを移動したデータ点があったかどうかを表す bool 値（`True` または `False`）

再配置の結果である新しいクラスタ集合は，これまでと同様に，クラスタを座標 `(x, y)` のリストで表して，クラスタのリストとして返しなさい（リストの $i$ 番目が $i$ 番目のクラスタ）．

K-means アルゴリズム全体は，データ点の再配置でクラスタが変化する限り繰り返しを続ける．すなわちデータが異なるクラスタへ移動することがあった場合は再び重心の計算と再配置を行う．

そのため，1回の `reallocate` の実行の間にクラスタ間でデータの移動があったかどうかを調べ，再配置結果のクラスタ集合と一緒に関数から返す．データの移動があった場合は `True`，無かった場合は `False` の値を取る変数を `change` とすれば，それを再配置結果のクラスタのリスト `new_clusters` と同時に返すには
```python
return [new_clusters, change]
```
のようにリストに両方を入れて返せばよい．呼び出し側では
```python
new_clusters, change = reallocate(centers, clusters)
```
のように2つに分けて受け取ることができる．

入出力例1：
```python
centers  = [(45.5, 45.5), (1, 1)] # 現在の重心

clusters = [ [(1, 1), (90, 90)],  # 現在のクラスタ 1
             [(0, 0),   (2, 2)] ] # 現在のクラスタ 2

--> 出力：[ [ [(90, 90)],                 # 再配置後のクラスタ 1
             [(1, 1), (0, 0), (2, 2)] ], # 再配置後のクラスタ 2
            True ] # 変化があったので change = True
```

入出力例2：
```python
centers  = [(0.5, 0.5), (2, 2)] # 現在の重心

clusters = [ [(0, 0), (1, 1)],  # 現在のクラスタ 1
             [(2, 2), (2, 2)] ] # 現在のクラスタ 2

--> 出力：[ [ [(0, 0), (1, 1)],   # 再配置後のクラスタ 1
             [(2, 2), (2, 2)] ], # 再配置後のクラスタ 2
            False ] # 変化が無かったので change = False
```

In [ ]:
def reallocate(centers, clusters):
    # *** 実装しなさい ***


ヒント（ダブルクリックで表示）
<!--
* まず再配置後のクラスタたちを表す長さ K のリスト new_clusters を

  new_clusters = [[], [], ..., []]

  と K 個の空リストで初期化する

* 変化があったかどうかを記録する変数 changes を False で初期化する

* 各クラスタ clusters[i] の各点 v について（-> ２重ループになる）

  * find_closest を使って最も近い重心をもつクラスタの番号 j を得る

  * new_clusters[j] に点 v を追加する

  * その際，j が現在のクラスタの番号と異なっていたら，つまり i != j だったら
    変数 changes を True にする

   （一つでもクラスタを移動した点があったら changes は True としたいので，
     i == j だったら changes = False とするのではなく，
    その場合は change を変化させない）

* 最後に [new_clusters, changes] を return する
-->

実装できたらテストしなさい（出力のクラスタ内の順番は以下のものと同一でなくてもよいが，素直に実装すればこの順番になるでしょう）：

In [ ]:
# 上の入出力例 1
centers1 = [(45.5, 45.5), (1, 1)] # 現在の重心

old_clusters1 = [ [(1, 1), (90, 90)], # 現在のクラスタ 1
                  [(0, 0), (2, 2)] ]  # 現在のクラスタ 2

new_clusters1 = [ [(90, 90)],                # 再配置後のクラスタ 1
                  [(1, 1), (0, 0), (2, 2)] ] # 再配置後のクラスタ 2

# 変化があったので change = True
print(reallocate(centers1, old_clusters1) == [new_clusters1, True])


# 上の入出力例2
centers2 = [(0.5, 0.5), (2, 2)] # 現在の重心

old_clusters2 = [[(0, 0), (1, 1)],  # 現在のクラスタ 1
                 [(2, 2), (2, 2)]]  # 現在のクラスタ 2

new_clusters2 = [[(0, 0), (1, 1)],  # 再配置後のクラスタ 1（変化なし）
                 [(2, 2), (2, 2)]]  # 再配置後のクラスタ 2（変化なし）

# 変化がないので change = False
print(reallocate(centers2, old_clusters2) == [new_clusters2, False])

### Step 3: メイン処理
ここまで実装した部品を組み合わせて，K-means アルゴリズム全体を関数として実装しよう．

入力のデータ点のリスト `data = [(x1, y1), (x2, y2), ...]` と，出力するクラスタの数 `K` の数を受け取り，データが移動しなくなるまで重心の計算とクラスタ間のデータの再配置（移動）を繰り返す関数 `k_means(data, K)` を実装しなさい．

出力として，結果のクラスタのリスト `clusters` と，それぞれの重心のリスト `centers` を<font color="red">まとめて `[clusters, centers]` の形で返しなさい．</font>

参照しやすいよう，もう一度，手順と対応する関数名を箇条書きしておく

0. 入力データをクラスタ群 $C_1, \dots, C_K$ にランダムに分割する（`random_split(data, K)`）
1. 各クラスタについて，その重心 $c_1, \dots, c_K$ を計算する（各クラスタに対して `mean(cluster)`）
2. 各入力データ `v` を，最も近い重心をもつクラスタに移動させる（`reallocate(centers, clusters)`）
3. もしもクラスタ間でのデータの移動があったら 1. に戻る．移動がなければアルゴリズム終了（`k_means(data, K)`）


ヒント（ダブルクリックで表示）
<!--
* 変化がある間は何かを繰りかえす, というタイプの手続きなので
  while 文を使うのが便利である．

* 全体としては, だいたい以下のような構造になるだろう

  data をランダムに分割した結果を clusters とする

  change = True # 最初に while ループに入るために True で初期化する

  while change: # 変化がある間くり返す
      clusters のそれぞれの重心を計算し, それを centers とする
      clusters を再配置の結果で上書きし, 変化があったかどうかを change にセットする

  clusters と centers を並べた長さ2のリストを return する
-->

In [ ]:
def k_means(data, K):
    # *** 実装しなさい ***


実装できたら，最初のセクションで作ったデータ `data2d` を入力とし，`K = 4` として実行して結果を見てみよう：

In [ ]:
clusters, centers = k_means(data2d, 4)
show_2d_clusters(clusters, centers)

上のセルを Control + Enter で何度か実行して，その度に結果が異なることを確認しなさい．

結果が異なるのは，最初にランダムにクラスタを作っており，最終結果もそれによって変化するためである．

何度も実行すると，ときどき点の色が3色しかない結果が見られるだろう．これは途中でどこかのクラスタが空になってしまった場合である．

入力データを生成したときの分布に従った色分けは以下のようだった：

In [ ]:
show_2d_clusters(true_clusters)

「グループ分け（色分け）のしかた」が問題なので，元の分布とクラスタリング結果で「同じ色（例えば赤）のクラスタが同じような場所にあるか」は問題ではない．
「グループ分けのしかた」は（最後に4つのクラスタがある場合）だいたい合っているだろうか．

#### 練習10.1：クラスタの数が合っていない場合
上ではデータを生成したときの「真のクラスタの数」と同じ `K=4` を用いて実行してみた．これをより少なくしたり（例えば `K=2` や `K=3`），多くしたら（例えば `K=10`）どうなるかやってみなさい．特に `K=2` のときに，Control + Enter で何度も実行すると，結果が大幅に変わることがあるかもしれない（なぜそうなるか？）

In [ ]:
# 最初に作った data2d を入力として，
# K = 2 あるいは K = 3 で K-means アルゴリズムを実行し, 結果を表示しなさい


In [ ]:
# 最初に作った data2d を入力として，
# 4 より大きな K （例えば K = 10）で K-means アルゴリズムを実行し, 結果を表示しなさい


<!--
#### 練習10.2：K-means アルゴリズムの処理過程の表示
アルゴリズムの動きをみるために, k_means` の各繰り返しの先頭で, その時点のクラスタ群を `show_2d_clusters` で表示するように変更し, 実行してみなさい．
ランダムな配置からどのようにクラスタが出来ていくか分かるだろう．
-->

---
## 課題10.3：ベクトル量子化
クラスタリングの応用として，１つの画像の中の色情報をクラスタリングしてみよう．

### 画像の読み込み
<!--
まず適当な画像をダウンロードしてこのノートブックと同じフォルダに置きなさい．Safari の場合は画像を右クリックし「イメージを別名で保存」を選択し，ノートブックと同じフォルダに適当な名前をつけて保存すればよい．
* イラストではなく写真のほうがクラスタリングの効果が分かりやすい
* あまり小さい画像ではクラスタリングの効果が分からない
* かと言ってあまり大きい画像は処理時間がかかりすぎるので，1辺が300ピクセル程度のものにしておくのがよい
* [Google 画像検索](https://www.google.co.jp/imghp?hl=ja)で検索結果の画面の「ツール」→「サイズ」→「アイコン」を選ぶとそれくらいのサイズの画像が見つけやすい

画像をノートブックと同じフォルダに置いたら，下のセルの <ファイル名> のところを画像ファイルの名前に書き換えて実行しなさい．
* <ファイル名> には，.jpg や .png のような拡張子まで含めること
* <ファイル名> を囲む `"` はそのままにすること．例えば `file = "レッサーパンダ.png"` のようにする．
-->
最初に [ここ](https://upload.wikimedia.org/wikipedia/commons/4/4e/Peafowl_head_at_Gappo_park_20060828.jpg) をクリックして画像を表示し，画像を右クリックしてこのノートブックと同じフォルダに保存しなさい．ファイル名は変えないこと．

画像ファイルは日本語版 Wikipedia の[クジャク](https://ja.wikipedia.org/wiki/クジャク)のページに[頭部の様子](https://ja.wikipedia.org/wiki/ファイル:Peafowl_head_at_Gappo_park_20060828.jpg)として掲載されているものである（[CC BY-SA 3.0](https://creativecommons.org/licenses/by-sa/3.0/legalcode)ライセンス）

ダウンロード後に以下のセルを実行すると画像が読み込まれる（PIL というライブラリを用いている．下のセルの詳細について今理解する必要はない）．

In [ ]:
from PIL import Image

file = "Peafowl_head_at_Gappo_park_20060828.jpg"
img = Image.open(file).convert("RGB")
img

画像が表示されれば読み込みは成功である．

### 画像データとピクセル
まず画像データの構造について，少し説明する．

画像データは，「ピクセル」と呼ばれるマス目が縦横に並んでできている．

下のセルを実行すると，上で読み込んだ画像の横方向のピクセルの数（width）と，縦方向のピクセルの数（height）が表示される：

In [ ]:
img.width, img.height

各ピクセルは一つの色で塗られている．
つまり画像ファイルは方眼紙のマスをいろいろな色で塗り分けたような構造になっている．

下のセルを実行すると，画像の中から8ピクセル x 8ピクセルの正方形を取り出して拡大した図が表示される．正方形の位置は，画像全体の左から `x` 番目，上から `y` 番目のピクセルが左上の隅となる位置で，元の画像の上に赤の枠で表示される（最初の状態では頭の上の方の羽の目玉模様のところに当たる．小さいので赤い点にしか見えないかもしれない）．

下のセルの `x` と `y` の値をいろいろ変えて試してみなさい．

（セルの中のプログラムについては，ここで理解する必要はない）

In [ ]:
import matplotlib.patches as patches
import numpy as np

x = 450 # ここを変更する
y = 240 # ここを変更する

t = 8

# 拡大して表示する範囲を, 赤い四角で画像の上に重ねて示す
plt.imshow(np.array(img))
plt.plot([x, x, x + t, x + t, x], [y, y + t, y + t, y, y], color="red")
plt.show()

# ピクセルと同じ色の四角を並べて拡大図を描く
ax = plt.axes()
ax.set_ylim([t, 0])
for xx in range(0, t):
    for yy in range(0, t):
        r, g, b = img.getpixel((x + xx,y + yy))
        fc = f"#{r:02x}{g:02x}{b:02x}"
        ax.add_patch(patches.Rectangle(xy=(xx, yy), width=1, height=1, fc=fc, ec='#000000'))
plt.axis('scaled')
ax.set_aspect('equal')
plt.show()

# ピクセルの RGB 値を並べて書く
for yy in range(0, t):
    for xx in range(0, t):
        r, g, b = img.getpixel((x + xx,y + yy))
        print("({:3d},{:3d},{:3d})".format(r, g, b), end="")
    print()

---
ピクセルの拡大図の下には，各セルの色の情報を (R, G, B) の形式で，拡大図のピクセルの配置どおりに並べて示している．

ここで (R, G, B) とは，光の3原色である赤（Red），緑（Green），青（Blue）のそれぞれの強さを 0～255 の範囲の整数で表したものである．

下のセルを実行すると，変数 `rgb` にセットされたタプル `(R, G, B)` の値を `(赤の強さ(R), 緑の強さ(G), 青の強さ(B))` とする色で塗った正方形が表示される．R, G, B の値を（0～255の範囲で）いくつか変えてみて，どのような色になるか見てみなさい（プログラムの中身自体をここで理解する必要はない）．

In [ ]:
import matplotlib.patches as patches

rgb = (255, 90, 127) # ここを変える

fc = "#{:02x}{:02x}{:02x}".format(*rgb)
rect = patches.Rectangle(xy=(0, 0), width=1, height=1, fc=fc)

plt.gca().add_patch(rect)
plt.axis('scaled')
plt.gca().set_aspect('equal')
plt.show()

---
### 画像の縮小

課題で用いるクジャクの画像は大きいため，そのままだと処理にだいぶ時間がかかる．
そこでまず，画像の縮小処理を行う．

最初に，元の画像の各ピクセルのRGB値を，画像上の位置に従って並べたデータ `px_matrix` を作る．
すなわち，画像の上から `i` ピクセル目，左から `j` ピクセル目のRGB値を `(Rij, Bij, Gij)` とすると，
`px_matrix` は以下のような2重リストである：
```python
[ [(R11, G11, B11), (R12, G12, B12), ..., (R1w, G1w, B1w)], # 画像の一番上の水平ライン
  [(R21, G21, B21), (R22, G22, B22), ..., (R2w, G2w, B2w)], # 画像の2番目の水平ライン
  ...
  [(Rh1, Gh1, Bh1), (Rh2, Gh2, Bh2), ..., (Rhw, Ghw, Bhw)] ] # 画像の一番下の水平ライン
```
ここで `h` と `w` とは画像の縦と横のピクセル数である．

下のセルをクリックして `px_matrix` を作りなさい（下のセルの中身について今は理解する必要はない）：

In [ ]:
px_matrix = [ [tuple(map(int, p)) for p in row] for row in np.array(img) ]

px_matrix の左上の 8 x 8 ピクセルの部分を表示してみよう：

In [ ]:
for i in range(8):
    for j in range(8):
        pixel = px_matrix[i][j] # 上から i 番目，左から j 番目のピクセル
        # 各ピクセルを (R, G, B) の形で表示
        print("({:3d},{:3d},{:3d})".format(pixel[0], pixel[1], pixel[2]), end="")
    print() # 改行

次に，画像サイズを縦に 1/s，横に 1/t にするために，`(R, G, B)` の２重リストで表された画像 `img` のピクセルを，縦に s 個おき，横に t 個おきに抜き出して並べた２重リストを作成して返す関数 `downsample(img, s, t)` を実装しなさい．

つまり入力 `img` に対して `downsample(img, s, t)` は以下のような２重リストになる：
```python
[ [img[  0][0], img[  0][t], img[  0][2*t], img[  0][3*t], ..., img[  0][T]],
  [img[1*s][0], img[1*s][t], img[1*s][2*t], img[1*s][3*t], ..., img[1*s][T]],
  [img[2*s][0], img[2*s][t], img[2*s][2*t], img[2*s][3*t], ..., img[2*s][T]],
  ...
  [img[ S ][0], img[ S ][t], img[ S ][2*t], img[ S ][3*t], ..., img[ S ][T]] ]
      
```
ここで `S` は `s` の倍数で画像の縦のサイズを超えない最大のもの，`T` は `t` の倍数で画像の横のサイズを超えない最大のものとする．

それでは `downsample(img, s, t)` を実装しなさい：

In [ ]:
def downsample(img, s, t):
    # 実装しなさい


ヒント（ダブルクリックで表示）
<!--
次の手順でできる

---
* まず出力 output を空のリストで初期化する

* i = 0, s, 2*s, ... のそれぞれについて

  * 出力の i 行目になるリスト row を空のリストで初期化する

  * j = 0, t, 2*t, ... のそれぞれについて
    * img[i][j] を row に追加する

  * row を output に追加する

* output を return する
---

- "i = 0, s, 2*s, ... のそれぞれについて" のところは

     for i in range(初期値, 上限+1, ステップ幅):

  の形式の range を使った for 文でできる.
  "j = 0, t, 2*t, ..." も同様．

- 画像の縦のサイズは len(img), 横のサイズは len(img[0]) で分かる
-->

実装できたら縦横それぞれ 1/10　にした画像を表示して，テストしなさい．解像度が低くなったのが分かるはずである：

In [ ]:
plt.imshow(downsample(px_matrix, 10, 10));

また（以下の課題では必要ないが）縦・横の縮小率を変えて，縦 1/10，横 1/3 の場合に結果が横長になることを確認しなさい：

In [ ]:
plt.imshow(downsample(px_matrix, 10, 3));

---
### クラスタリングによる画像の圧縮

例えば 1000ピクセル x 1000ピクセル の画像は，各ピクセルのRGBの値がそれぞれ1バイト（= 8ビット = 0～255 の $2^8$ 通りの値）を使用するため，
何の圧縮もしなければ
```
1000 × 1000 × 3バイト = 3メガバイト
```
のデータ量になる．

この画像中の100万個のピクセルを，似た色のものをまとめることで8つのクラスタに分けたとする．
このとき，各クラスタに属するピクセルを，その重心が表す色に塗り替えることを考える．
つまり，似たような色のピクセルはまとめて同じ色（それらの平均の色）で塗りつぶしてしまうということ．

もちろん色を8色に限定することで，画像としては劣化するが，各ピクセルが8色中のどの色かは
3ビット（→ $2^3 = 8$ 通り）で表現できる．そのため，データ量はおよそ
```
1000 × 1000 × 3ビット = 約400キロバイト
```
と約1/8になる．

このような画像圧縮の方法を**ベクトル量子化**という．
現在の動画や画像フォーマットでも，その他の圧縮手法と組合せて実際に使われている．

以下，ベクトル量子化を K-means アルゴリズムを使って実装し，色情報の圧縮の結果がどのような画像になるか見てみよう．手順は以下のようになる


0. まず画像を適当な大きさに縮小する．
1. 次に，画像の全ピクセルを，`[(R1, G1, B1), (R2, G2, B2), ...]` のように，各ピクセルの `(R, G, B)` の値を並べたリスト `pixels` として表す．
2. つぎに，`pixels` の中の `(R, G, B)` を3次元のベクトルとみなし，K-means アルゴリズムによってクラスタリングする．
3. 最後に，各クラスタに属するピクセルの色を，そのピクセルが属するクラスタの重心に対応する色で塗り替える．

クジャクの画像を縦 1/3, 横 1/3 にしたデータ `px_matrix_mini` を作りなさい：

In [ ]:
px_matrix_mini = downsample(px_matrix, 3, 3)

次に px_matrix_mini の各行を連結して一つのリストしたデータ `pixels` を作りなさい：

In [ ]:
pixels = sum(px_matrix_mini, [])

注：`sum(２重リスト, [])` で連結ができる理由（ダブルクリックで表示）．
<!--
数値のリスト `xs = [1, 2, 3]` に関数 `sum` を適用すると

sum(xs) = 1 + 2 + 3 = 6

という計算が行われる．`sum` には第２引数として初期値を与えることができ，
たとえば初期値として `10` を与えれば

sum(xs, 10) = 10 + 1 + 2 + 3 = 16

となる．

この形の `sum` を利用すると，例えば `yss = [["a", "b"], ["x", "y"]]` という２重リストに対し
初期値 `[]` を指定して `sum` を呼び出すことで

sum(yss, []) = [] + ["a", "b"] + ["x", "y"] = ["a", "b", "x", "y"]

となり，２重リストの各要素のリストを連結した結果が得られる．
-->

画像の各行を連結したリスト `pixels` の最初のほうの要素を見てみなさい（画像の左上隅のピクセルの値に対応する）：

In [ ]:
pixels[0:10]

次に，`pixels` の中の3次元ベクトルをクラスタリングするために，最初のセクションで2次元ベクトルのクラスタリング用に実装した K-means アルゴリズムのプログラムを修正しなさい．

3次元ベクトルに対応するためには，以下の2つの関数を修正するだけで良いはずである：
* クラスタ `cluster` の平均を計算する関数 `mean(cluster)` <!-- <a href="#Step-1:-重心の計算">[2次元版の実装に飛ぶ]</a> -->
* クラスタの重心のリスト `centers` のうち，点 `v` に最も近いものの番号を返す関数 `find_closest(centers, v)` <!-- <a href="#Step-2:-クラスタへの再配置">[2次元版の実装に飛ぶ]</a> -->

上の2つの関数に加え，メインの関数 `k_means` の引数に，最大の繰り返し回数 `max_iter` を追加し，`k_means(data, K, max_iter)` と修正しよう．つまり，まだデータがクラスタ間を移動することがある場合でも，重心の計算とデータの再配置の繰り返し回数が `max_iter` に到達したらそこで処理を止め，その時点でのクラスタ集合と重心を返すようにする．これは，何万個ものピクセルをクラスタリングする場合，データの移動がなくなるまでには非常に長い時間がかかるためである．<!-- <a href="#Step-3:-メイン処理">[2次元版の実装に飛ぶ]</a> -->

以上，修正点をまとめると
* `mean(cluster)` → 3次元ベクトルを処理するように変更
* `find_closest(centers, v)` → 3次元ベクトルを処理するように変更
* `k_means(data, K, max_iter)` → 最大繰り返し回数 `max_iter` に到達したらそこで終わるように変更

2次元ベクトル版の実装を**直接変更せずに**，いったん下のセルに必要な関数をコピーしてから，それを修正しなさい：

In [ ]:
# *** 2次元ベクトル版の mean, find_closest, k_means をコピーして
# *** 必要な修正を加えなさい


必要な修正が出来たら，`pixels` の中の3次元ベクトルに対して `K=8`，`max_iter=5` として `k_means` を適用してみなさい（1分くらいかかるかもしれない）：

In [ ]:
K = 8
max_iter = 5

_, centers = k_means(pixels, K, max_iter) # 返り値のうち clusters は使わない

クラスタリングが終わったら，各クラスタの重心に対応するRGB値を並べたリスト `palette` を作成する．
これを用いて，元の画像のピクセル値を `palette` の中の色に置き換える．

注：重心の座標そのものは浮動小数点数であり，画像のRBG値の各成分は整数とする必要があるため，以下のセルで整数に丸める処理をしている．

以下のセルを実行して，リスト `palette` に値をセットしなさい：

In [ ]:
def make_palette(centers):
    palette = []
    for r, g, b in centers: # 重心をひとつずつ取り出す
        if r == float("inf"):
            # 空のクラスタに対応 -> 色は何でもよい
            palette.append((0, 0, 0))
        else:
            # 四捨五入して整数にする
            palette.append((int(round(r)), int(round(g)), int(round(b))))
    return palette
            
palette = make_palette(centers)

`palette` の中身を見てみよう：

In [ ]:
palette

ピクセルの `(R, G, B)` を並べた２重リスト `img` の各ピクセルに対し，最も近い色を `palette` 中から探し，`palette` の色に置き換えた２重リストとして，ベクトル量子化した画像データを作る．

例えば `img` が以下の 2x2 ピクセルの画像データだったとする
```python
img = [[(10, 10, 10), (12, 12, 10)],
       [(90, 90, 98), (11, 11, 10)]]
```
これに対し，以下の２色の `palette` を用いてベクトル量子化したとする
```python
pallett = [(11, 11, 11), (99, 99, 88)]
```
そのときに結果は以下のようになるはずである：
```python
ベクトル量子化の結果：
[[(11, 11, 11), (11, 11, 11)],  # <- ２つとも palette の 0 番目の色に置き換わった
 [(99, 99, 88), (11, 11, 11)]]  # <- 左のピクセルは 1 番目の色，右のピクセルは 0 番目の色に置き換わった
```

`img` および `palette` を受け取り，上の処理を行った結果を返す関数 `quantize(img, palette)` を実装しなさい．

**注意**：与えられた `img` の中身を**置き換えるのではなく**，色を置き換えた結果を新しい２重リストとして作成して返すこと．

In [ ]:
# 入力:
#   img ... 画像のピクセル情報を並べた2重リスト
#   palette ... クラスタの重心を整数のRGB値に変換したリスト
# 出力:
#   img 中の各RGB値を，それに最も近い pallett 中の色で
#   置き換えた (R, G, B) の2重リスト
def quantize(img, palette):
    # *** 実装しなさい ***


ヒント（ダブルクリックで表示）
<!--

img の中のあるピクセル p = (R, G, B) に対し，palette の中で最も近い色を探す処理は
K-means クラスタリングで使った find_closest の処理と同じなので
 
k = find_closest(palette, p)

で最も近い色の添字 k が取得できる．よって p は palette[k] に置き換えればよい．

-->

実装できたら，`quantize` の結果を元画像と並べて見てみよう：

In [ ]:
quantized = quantize(px_matrix_mini, palette)

_, ax = plt.subplots(1, 2, figsize=(16, 32))
ax[0].set_title("Original")
ax[0].imshow(np.array(img))
ax[1].set_title("Quantized")
ax[1].imshow(quantized);

「Quantized」のほう（ベクトル量子化の結果）は細かい色の変化が無くなり，だいたい8色くらいで描かれているように見えれば多分できている．

（クジャクであることは分かるが，それほどきれいな画像にはならない）

### 練習10.4: ベクトル量子化全体の処理の関数化
以下のセルの `"実装しなさい"` の部分を，ここまで実装した関数を利用して埋め，画像の読み込みから結果の表示までの一連の流れを関数としてまとめなさい．

In [ ]:
import sys

# 入力:
#   file ... 画像ファイル名
#   s ... 縮小率（整数）: 縦横 1/s に縮小
#   K ... クラスタ数
#   max_iter ... 最大くり返し数
def vec_quantize(file, s, K, max_iter):
    # 画像の読み込み
    img = Image.open(file).convert("RGB")
    # (R, G, B) の２重リストに変換
    px_matrix = [ [tuple(map(int, p)) for p in row] for row in np.array(img) ]
    
    # ** 実装しなさい ***
    # - px_matrix を縦横 1/s に縮小した２重リスト px_matrix_mini を作成する
    # - 縮小後の画像のピクセルを一列に並べたリストを作成する
    # - クラスタ数 K, 最大繰り返し数 max_iter で K-means クラスタリングを行う
    # - クラスタの重心に対応する色のリスト palette を作成する
    # - 縮小した画像 px_matrix_mini の各ピクセルを
    #   palette 中の色で置き換えた画像 quantized を作成する
    # - 元画像と並べて表示する（この部分は下に実装済み）
    
    # 結果の表示
    _, ax = plt.subplots(1, 2, figsize=(16, 32))
    # 元画像を表示
    ax[0].set_title("Original")
    ax[0].imshow(np.array(img))
    # ベクトル量子化した画像を表示
    ax[1].set_title("Quantized")
    ax[1].imshow(quantized)
    plt.show();

実装できたら下のセルをクリックし，K = 2, 4, 8, 16 それぞれの結果を元画像と並べて表示しなさい．

In [ ]:
filename = "Peafowl_head_at_Gappo_park_20060828.jpg"
scale = 3 # 縦横 1/3 に縮小
max_iter = 5 # 最大繰り返し数はいつも 5 

for k in [2, 4, 8, 16]:
    print("K =", k)
    vec_quantize(filename, scale, k, max_iter)

<!--
**発展課題**: このセクションで行ったベクトル量子化の実装では，最後に画像の各ピクセルに最も近いクラスタ重心を `centers` から探しているが，
本当は各ピクセルがどのクラスタに属すかは既に分かっているので無駄である．
具体的には `k_means` の繰り返し一回分くらいの無駄である．
この無駄を取り除いてより効率的な実装にせよ．
-->

---
お疲れ様でした．以上で今回の課題は終わりです．

## 課題提出の前の注意
* かならずメニューの "Run" から "Run All Cells" を選択し，ここまでの全てのセルが正しく実行されることを確認すること
* "Run All Cells" を実行したら，各セルの実行結果が表示されている状態で保存のボタンを押してノートブックを保存すること
* 上記のようにして，実行結果まで含めて保存してからノートブックを提出すること．
* <font color="red">実行結果が保存されていないノートブックは採点しません．</red>

中級編１：おわり